In [1]:
import pandas as pd
import os
import re
import requests
from bs4 import BeautifulSoup

In [2]:
data = pd.read_csv('final.csv', low_memory=False)

In [3]:
drop_columns = ['ActivityDepthHeightMeasure/MeasureValue', 
                'ActivityDepthHeightMeasure/MeasureUnitCode', 
                'ActivityDepthAltitudeReferencePointText', 
                'ActivityTopDepthHeightMeasure/MeasureValue', 
                'ActivityTopDepthHeightMeasure/MeasureUnitCode', 
                'ActivityBottomDepthHeightMeasure/MeasureValue',
                'ActivityBottomDepthHeightMeasure/MeasureUnitCode',
                'ActivityLocation/HorizontalCollectionMethodName',
                'ActivityLocation/HorizontalCoordinateReferenceSystemDatumName',
                'PassCount',
                'SampleCollectionMethod/MethodIdentifier',
                'SampleCollectionMethod/MethodIdentifierContext',
                'SampleCollectionMethod/MethodName',
                'SampleCollectionMethod/MethodQualifierTypeName',
                'SampleCollectionMethod/MethodDescriptionText',
                'SampleCollectionEquipmentName',
                'SampleCollectionMethod/SampleCollectionEquipmentCommentText',
                'SamplePreparationMethod/MethodIdentifier',
                'SamplePreparationMethod/MethodIdentifierContext',
                'SamplePreparationMethod/MethodName',
                'SampleContainerTypeName',
                'SampleContainerColorName',
                'ThermalPreservativeUsedName',
                'SampleTransportStorageDescription',
                'ActivityMetricUrl',
                'ProviderName',
                'ActivityMetricType/MetricTypeIdentifier',
                'ActivityMetricType/MetricTypeIdentifierContext',
                'MetricTypeCitation/ResourceTitleName',
                'MetricTypeCitation/ResourceCreatorName',
                'MetricTypeCitation/ResourceSubjectText',
                'MetricTypeCitation/ResourcePublisherName',
                'MetricTypeCitation/ResourceDate',
                'MetricTypeCitation/ResourceIdentifier',
                'MetricTypeCitation/FormulaDescriptionText',
                'AnalysisStartTime/Time',
                'AnalysisStartTime/TimeZoneCode',
                'ElectronicAddress',
                'Telephonic',
                'OrganizationAddress/AddressTypeName_1',
                'OrganizationAddress/AddressText_1',
                'OrganizationAddress/SupplementalAddressText_1',
                'OrganizationAddress/LocalityName_1',
                'OrganizationAddress/StateCode_1',
                'OrganizationAddress/PostalCode_1',
                'OrganizationAddress/CountryCode_1',
                'OrganizationAddress/CountyCode_1',
                'OrganizationAddress/AddressTypeName_2',
                'OrganizationAddress/AddressText_2',
                'OrganizationAddress/AddressText_2',
                'OrganizationAddress/LocalityName_2',
                'OrganizationAddress/StateCode_2',
                'OrganizationAddress/PostalCode_2',
                'OrganizationAddress/CountryCode_2',
                'OrganizationAddress/CountyCode_2',
                'OrganizationAddress/AddressTypeName_3',
                'OrganizationAddress/AddressText_3',
                'OrganizationAddress/SupplementalAddressText_3',
                'OrganizationAddress/LocalityName_3',
                'OrganizationAddress/StateCode_3',
                'OrganizationAddress/PostalCode_3',
                'OrganizationAddress/CountryCode_3',
                'OrganizationAddress/CountyCode_3',
                'SamplingDesignTypeCode'
               ]
data = data.drop(columns=drop_columns)
data = data.dropna(subset=['ResultMeasureValue', 'CharacteristicName', 'OrganizationIdentifier', 'ActivityStartDate'])

In [4]:
df_ts = data[['OrganizationFormalName',
              'OrganizationIdentifier',
                    'ActivityStartDate', 
                    'CharacteristicName', 
                    'ResultSampleFractionText', 
                    'ResultMeasureValue', 
                    'ResultMeasure/MeasureUnitCode', 
                    'StatisticalBaseCode', 
                    'ResultValueTypeName', 
                    'ResultStatusIdentifier',
                    'MonitoringLocationIdentifier', 
                    # 'MonitoringLocationName', 
                    # 'ActivityLocation/LongitudeMeasure', 
                    'LongitudeMeasure', 
                    # 'ActivityLocation/LatitudeMeasure', 
                    'LatitudeMeasure']]

In [5]:
df_ts

,OrganizationFormalName,OrganizationIdentifier,ActivityStartDate,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/MeasureUnitCode,StatisticalBaseCode,ResultValueTypeName,ResultStatusIdentifier,MonitoringLocationIdentifier,LongitudeMeasure,LatitudeMeasure
0,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Stream flow,NaN,1900,ft3/s,Mean,Actual,Accepted,USGS-02336000,NaN,NaN
1,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Stream flow,NaN,54,m3/sec,Mean,Actual,Accepted,USGS-02336000,NaN,NaN
2,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Escherichia coli,NaN,3800,MPN/100 ml,NaN,Actual,Accepted,USGS-02336000,NaN,NaN
3,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Total Coliform,Total,130000,MPN/100 ml,NaN,Actual,Accepted,USGS-02336000,NaN,NaN
4,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Turbidity,Total,29,NTRU,NaN,Actual,Accepted,USGS-02336000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
522341,"USEPA, Office of Water, Office of Science and ...",OST_SHPD,2019-09-12,"2,3',5'-Trichlorobiphenyl",NaN,0.00297,ng/g,NaN,Actual,Final,OST_SHPD-NRS18_GA_10020,-84.82877,33.51301
522342,"USEPA, Office of Water, Office of Science and ...",OST_SHPD,2019-09-12,PCB-156/157,NaN,1.1,ng/g,NaN,Actual,Final,OST_SHPD-NRS18_GA_10020,-84.82877,33.51301
522343,"USEPA, Office of Water, Office of Science and ...",OST_SHPD,2019-09-12,"2,2',3,3',4,5',6-Heptachlorobiphenyl",NaN,0.169,ng/g,NaN,Actual,Final,OST_SHPD-NRS18_GA_10020,-84.82877,33.51301
522344,"USEPA, Office of Water, Office of Science and ...",OST_SHPD,2019-09-12,"2,2',3,5,6'-Pentachlorobiphenyl",NaN,0.0121,ng/g,NaN,Actual,Final,OST_SHPD-NRS18_GA_10020,-84.82877,33.51301


In [6]:
df_ts['OrganizationIdentifier'].unique()

array(['USGS-GA', '21GAEPD_WQX', 'CHATTAHOOCHEERIVERKEEPER',
       'R4ATHENS_WQX', '11NPSWRD_WQX', 'NALMS', 'NARS_WQX', 'OST_SHPD'],
      dtype=object)

In [7]:
USGS_GA = df_ts[df_ts['OrganizationIdentifier'] == 'USGS-GA']
USGS_GA

,OrganizationFormalName,OrganizationIdentifier,ActivityStartDate,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/MeasureUnitCode,StatisticalBaseCode,ResultValueTypeName,ResultStatusIdentifier,MonitoringLocationIdentifier,LongitudeMeasure,LatitudeMeasure
0,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Stream flow,NaN,1900,ft3/s,Mean,Actual,Accepted,USGS-02336000,NaN,NaN
1,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Stream flow,NaN,54,m3/sec,Mean,Actual,Accepted,USGS-02336000,NaN,NaN
2,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Escherichia coli,NaN,3800,MPN/100 ml,NaN,Actual,Accepted,USGS-02336000,NaN,NaN
3,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Total Coliform,Total,130000,MPN/100 ml,NaN,Actual,Accepted,USGS-02336000,NaN,NaN
4,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Turbidity,Total,29,NTRU,NaN,Actual,Accepted,USGS-02336000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426495,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Stream flow, instantaneous",NaN,0.00,m3/sec,NaN,Actual,Accepted,USGS-340388084232801,NaN,NaN
426496,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Temperature, water",NaN,21.6,deg C,NaN,Actual,Accepted,USGS-340388084232801,NaN,NaN
426497,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Stream flow, instantaneous",NaN,0.06,ft3/s,NaN,Actual,Accepted,USGS-340388084232801,NaN,NaN
426498,USGS Georgia Water Science Center,USGS-GA,2012-09-10,Specific conductance,Total,39,uS/cm @25C,NaN,Actual,Accepted,USGS-340388084232801,NaN,NaN


In [8]:
unique_values_df = pd.DataFrame(USGS_GA['MonitoringLocationIdentifier'].unique(), columns=['MonitoringLocationIdentifier'])
unique_values_df

,MonitoringLocationIdentifier
0,USGS-02336000
1,USGS-02336020
2,USGS-02336300
3,USGS-02336529
4,USGS-02336526
...,...
155,USGS-340397084196601
156,USGS-334925084273601
157,USGS-333253084392101
158,USGS-333361084360701


In [9]:
numbers = []
for line in unique_values_df['MonitoringLocationIdentifier']:
    number = re.findall(r'\d+', line)
    numbers.append(number[0])
unique_values_df['Number'] = numbers
unique_values_df

,MonitoringLocationIdentifier,Number
0,USGS-02336000,02336000
1,USGS-02336020,02336020
2,USGS-02336300,02336300
3,USGS-02336529,02336529
4,USGS-02336526,02336526
...,...,...
155,USGS-340397084196601,340397084196601
156,USGS-334925084273601,334925084273601
157,USGS-333253084392101,333253084392101
158,USGS-333361084360701,333361084360701


In [10]:
for index, row in unique_values_df.iterrows():
    site_number = row['Number']
    url = f"https://waterdata.usgs.gov/nwis/inventory/?site_no={site_number}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        dd_tags = soup.find_all('dd')
        for dd in dd_tags:
            if "Latitude" in dd.text and "Longitude" in dd.text:
                split_text = dd.text.split()
                
                latitude_string = ''.join(char for char in split_text[1] if not char.isalpha() or char.isdigit())
                degrees = int(latitude_string.split('°')[0])
                minutes = int(latitude_string.split('°')[1].split('\'')[0])
                seconds = float(latitude_string.split('°')[1].split('\'')[1].split('\"')[0])
                latitude = degrees + minutes / 60 + seconds / 3600
                
                longitude_string = ''.join(char for char in split_text[3] if not char.isalpha() or char.isdigit())
                degrees = int(longitude_string.split('°')[0])
                minutes = int(longitude_string.split('°')[1].split('\'')[0])
                seconds = float(longitude_string.split('°')[1].split('\'')[1].split('\"')[0])
                longitude = degrees + minutes / 60 + seconds / 3600
                
                unique_values_df.at[index, 'LatitudeMeasure'] = latitude
                unique_values_df.at[index, 'LongitudeMeasure'] = longitude
                break
    else:
        print(f"Failed to fetch data for site number: {site_number}")
unique_values_df

,MonitoringLocationIdentifier,Number,LatitudeMeasure,LongitudeMeasure
0,USGS-02336000,02336000,33.859167,84.454444
1,USGS-02336020,02336020,33.827222,84.455556
2,USGS-02336300,02336300,33.820306,84.407639
3,USGS-02336529,02336529,33.799167,84.486944
4,USGS-02336526,02336526,33.794472,84.474194
...,...,...,...,...
155,USGS-340397084196601,340397084196601,34.058881,84.317961
156,USGS-334925084273601,334925084273601,33.823611,84.460000
157,USGS-333253084392101,333253084392101,33.540703,84.664503
158,USGS-333361084360701,333361084360701,33.556278,84.613075


In [11]:
column_to_drop = 'Number'
unique_values_df.drop(column_to_drop, axis=1, inplace=True)
unique_values_df

,MonitoringLocationIdentifier,LatitudeMeasure,LongitudeMeasure
0,USGS-02336000,33.859167,84.454444
1,USGS-02336020,33.827222,84.455556
2,USGS-02336300,33.820306,84.407639
3,USGS-02336529,33.799167,84.486944
4,USGS-02336526,33.794472,84.474194
...,...,...,...
155,USGS-340397084196601,34.058881,84.317961
156,USGS-334925084273601,33.823611,84.460000
157,USGS-333253084392101,33.540703,84.664503
158,USGS-333361084360701,33.556278,84.613075


In [12]:
column_to_drop = ['LatitudeMeasure', 'LongitudeMeasure']
USGS_GA = USGS_GA.drop(column_to_drop, axis=1)
USGS_GA

,OrganizationFormalName,OrganizationIdentifier,ActivityStartDate,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/MeasureUnitCode,StatisticalBaseCode,ResultValueTypeName,ResultStatusIdentifier,MonitoringLocationIdentifier
0,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Stream flow,NaN,1900,ft3/s,Mean,Actual,Accepted,USGS-02336000
1,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Stream flow,NaN,54,m3/sec,Mean,Actual,Accepted,USGS-02336000
2,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Escherichia coli,NaN,3800,MPN/100 ml,NaN,Actual,Accepted,USGS-02336000
3,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Total Coliform,Total,130000,MPN/100 ml,NaN,Actual,Accepted,USGS-02336000
4,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Turbidity,Total,29,NTRU,NaN,Actual,Accepted,USGS-02336000
...,...,...,...,...,...,...,...,...,...,...,...
426495,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Stream flow, instantaneous",NaN,0.00,m3/sec,NaN,Actual,Accepted,USGS-340388084232801
426496,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Temperature, water",NaN,21.6,deg C,NaN,Actual,Accepted,USGS-340388084232801
426497,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Stream flow, instantaneous",NaN,0.06,ft3/s,NaN,Actual,Accepted,USGS-340388084232801
426498,USGS Georgia Water Science Center,USGS-GA,2012-09-10,Specific conductance,Total,39,uS/cm @25C,NaN,Actual,Accepted,USGS-340388084232801


In [13]:
result_df = pd.merge(USGS_GA, unique_values_df, on='MonitoringLocationIdentifier', how='inner')
result_df

,OrganizationFormalName,OrganizationIdentifier,ActivityStartDate,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/MeasureUnitCode,StatisticalBaseCode,ResultValueTypeName,ResultStatusIdentifier,MonitoringLocationIdentifier,LatitudeMeasure,LongitudeMeasure
0,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Stream flow,NaN,1900,ft3/s,Mean,Actual,Accepted,USGS-02336000,33.859167,84.454444
1,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Stream flow,NaN,54,m3/sec,Mean,Actual,Accepted,USGS-02336000,33.859167,84.454444
2,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Escherichia coli,NaN,3800,MPN/100 ml,NaN,Actual,Accepted,USGS-02336000,33.859167,84.454444
3,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Total Coliform,Total,130000,MPN/100 ml,NaN,Actual,Accepted,USGS-02336000,33.859167,84.454444
4,USGS Georgia Water Science Center,USGS-GA,2001-09-24,Turbidity,Total,29,NTRU,NaN,Actual,Accepted,USGS-02336000,33.859167,84.454444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345678,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Stream flow, instantaneous",NaN,0.00,m3/sec,NaN,Actual,Accepted,USGS-340388084232801,34.051911,84.397578
345679,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Temperature, water",NaN,21.6,deg C,NaN,Actual,Accepted,USGS-340388084232801,34.051911,84.397578
345680,USGS Georgia Water Science Center,USGS-GA,2012-09-10,"Stream flow, instantaneous",NaN,0.06,ft3/s,NaN,Actual,Accepted,USGS-340388084232801,34.051911,84.397578
345681,USGS Georgia Water Science Center,USGS-GA,2012-09-10,Specific conductance,Total,39,uS/cm @25C,NaN,Actual,Accepted,USGS-340388084232801,34.051911,84.397578


In [14]:
REST =  df_ts[df_ts['OrganizationIdentifier'] != 'USGS-GA']
REST

,OrganizationFormalName,OrganizationIdentifier,ActivityStartDate,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/MeasureUnitCode,StatisticalBaseCode,ResultValueTypeName,ResultStatusIdentifier,MonitoringLocationIdentifier,LongitudeMeasure,LatitudeMeasure
426500,Georgia DNR Environmental Protection Division,21GAEPD_WQX,1991-03-06,Turbidity,NaN,8,NTU,NaN,Actual,Final,21GAEPD_WQX-RV_12_4316,-84.40760,33.82040
426501,Georgia DNR Environmental Protection Division,21GAEPD_WQX,1991-03-06,Fecal Coliform,NaN,1700,MPN/100mL,NaN,Actual,Final,21GAEPD_WQX-RV_12_4316,-84.40760,33.82040
426502,Georgia DNR Environmental Protection Division,21GAEPD_WQX,1991-03-06,Flow,NaN,82,ft3/sec,NaN,Actual,Final,21GAEPD_WQX-RV_12_4316,-84.40760,33.82040
426503,Georgia DNR Environmental Protection Division,21GAEPD_WQX,1991-03-06,Ammonia-nitrogen,Dissolved,0.07,mg/L,NaN,Actual,Final,21GAEPD_WQX-RV_12_4316,-84.40760,33.82040
426504,Georgia DNR Environmental Protection Division,21GAEPD_WQX,1991-03-06,Phosphorus,Total,0.12,mg/L,NaN,Actual,Final,21GAEPD_WQX-RV_12_4316,-84.40760,33.82040
...,...,...,...,...,...,...,...,...,...,...,...,...,...
522341,"USEPA, Office of Water, Office of Science and ...",OST_SHPD,2019-09-12,"2,3',5'-Trichlorobiphenyl",NaN,0.00297,ng/g,NaN,Actual,Final,OST_SHPD-NRS18_GA_10020,-84.82877,33.51301
522342,"USEPA, Office of Water, Office of Science and ...",OST_SHPD,2019-09-12,PCB-156/157,NaN,1.1,ng/g,NaN,Actual,Final,OST_SHPD-NRS18_GA_10020,-84.82877,33.51301
522343,"USEPA, Office of Water, Office of Science and ...",OST_SHPD,2019-09-12,"2,2',3,3',4,5',6-Heptachlorobiphenyl",NaN,0.169,ng/g,NaN,Actual,Final,OST_SHPD-NRS18_GA_10020,-84.82877,33.51301
522344,"USEPA, Office of Water, Office of Science and ...",OST_SHPD,2019-09-12,"2,2',3,5,6'-Pentachlorobiphenyl",NaN,0.0121,ng/g,NaN,Actual,Final,OST_SHPD-NRS18_GA_10020,-84.82877,33.51301


In [15]:
result_df['LongitudeMeasure'] = -result_df['LongitudeMeasure']

In [16]:
df = pd.concat([result_df, REST], ignore_index=True)
df.to_csv('pretty_good.csv', index=False)

In [17]:
grouped = df.groupby('OrganizationIdentifier')

for org_id, group_df in grouped:

    folder_name = str(org_id)
    os.makedirs(folder_name, exist_ok=True)

    location_groups = group_df.groupby('MonitoringLocationIdentifier')

    for location_id, location_group_df in location_groups:
        
        file_name = f"{location_id}.csv"

        file_path = os.path.join(folder_name, file_name)

        location_group_df.to_csv(file_path, index=False)

print("Data has been divided and stored in folders based on OrganizationIdentifier and files based on MonitoringLocationIdentifier.")

Data has been divided and stored in folders based on OrganizationIdentifier and files based on MonitoringLocationIdentifier.
